In [17]:
import csv
import pandas as pd
df=pd.DataFrame(columns=["Tweet_index","Tweet_Text"]);
txt_file=r"SemEval2018-T3_input_test_taskA_emoji.txt"
#csv_file=r"data11.csv"
i=0
with open(txt_file) as f:
    for line in f:
        #print(line)
        if i==0:
            i=i+1
            continue
        df.loc[i]=[text for text in line.split("\t")]
        i=i+1
df.to_csv('input_data_TestA.csv', sep=' ',index=False)
tweets=df["Tweet_Text"]
print(len(tweets))

784


In [18]:
import emoji
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
input_txt=""
non_normalized_input_txt=""
for line in tweets:
    #print(line)
    non_normalized_input_txt+=line
    de_emojize=emoji.demojize(line)
    remove_multidots = re.sub('\.\.\.+','...',de_emojize)
    remove_verticalbars = re.sub(r'\|', '',remove_multidots)
    remove_multiplewhite=re.sub(' +',' ',remove_verticalbars)
    commonurl_str = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', 'http://someurl', remove_multiplewhite)
    replace_username=re.sub('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','@someuser',commonurl_str)
  #  puncutation_flooding=re.sub(r'['+string.punctuation+']+', ' ',replace_username)
    input_txt+=replace_username
#print(input_txt)
input_tokenized=nltk.word_tokenize(input_txt)
stop_words = set(stopwords.words('english'))

#print(input_tokenized)
input_posTag=nltk.pos_tag(input_tokenized)
#print(input_posTag)
print(non_normalized_input_txt)

@Callisto1947 Can U Help?||More conservatives needed on #TSU + get paid 4 posting stuff like this!||YOU $ can go to http://t.co/JUmMWi0AyT
Just walked in to #Starbucks and asked for a "tall blonde" Hahahaha
GONNA WIN http://t.co/Mc9ebqjAqj
@mickymantell He is exactly that sort of person. Weirdo!
So much at work mate 10/10 #boring 100% #dead mate full on #shit absolutely #sleeping mate can't handle the
Corny jokes are my absolute favorite
People complain about my backround pic and all I feel is like "hey don't blame me, Albert E might have spoken those words" #life
@SR_Duncan @benpobjie Darn, my sock joke needs fixing?
if Christian expects Fifa to sleep in my bed with me tonight, he's wrong 👿
People who tell people with anxiety to "just stop worrying about it" are my favorite kind of people #educateyourself
Most important thing I've learned in school http://t.co/TvfJMGBlBh
#OrionLaunch underway - yeah! #Orion http://t.co/0Qkv1yq8T0
I love context and large ensemble Fridays!!!!! Der my m

In [19]:
def punct_flooding(str):
    l = len(str)
    count=0
    for i in range(l):
        cur_count=1
        if str[i] in string.punctuation:
            for j in range(i+1,l):
                if str[j] not in string.punctuation or str[j] in ['/','#','.']:
                    break
                else:
                    cur_count+=1
        if cur_count > count: 
            count = cur_count 
    if count >2:
        return 1
    else:
        return 0

In [20]:
def char_flooding(str):
    l = len(str)
    count=0
    for i in range(l):
        cur_count=1
        if str[i].isalpha and str[i] not in string.punctuation and not str[i].isnumeric :
             for j in range(i+1,l):
                if not str[j] == str[i] :
                    break
                else:
                    cur_count+=1
        if cur_count > count: 
            count = cur_count 
    if count >2:
        return 1
    else:
        return 0

In [21]:
def punct_present(str):
    l = len(str)
    cur_count=0
    for i in range(l):
        if str[i] in string.punctuation and str[i] not in ['/','#','.'] :
            cur_count=1
            break;
    return cur_count

In [22]:
import pandas as pd
import re
import emoji
lexfeatures=[]
i=0
for line in input_txt.split('\n'):
    if(len(line)>0):
        hashCount=line.count("#")
        ispunctflooded= punct_flooding(line)
        ispunctpresent= punct_present(line)
        #print(ispunctpresent)
        ischarflooded=char_flooding(line)
        emojislist=re.findall(r':[\w]+:',line)
        emojiCount=len(emojislist)
        tweetLength=len(line)
        wordsList= line.split(' ')
        numberOfWords=len(wordsList)
        hashtagWordRatio= round(hashCount/numberOfWords,2)
        #print(emojislist)
        #print(emojiCount)
        lexfeatures.insert(i,{'Punctuation': ispunctpresent,'Character_Flooding':ischarflooded,'Punctutation_Flooding':ispunctflooded,'Hashtag_Frequency':hashCount,'Hashtag_to_word_ratio':hashtagWordRatio,'Emoticon_Count':emojiCount,'Tweet_length':tweetLength})
        i=i+1
df = pd.DataFrame(lexfeatures)
df.to_csv('outfiles/lexFeatures_TestA.csv', sep=',',index=False)
#print(df)

In [7]:
def getCount(dict11):
    dict13={}
    for key, value in dict11.items():
        if value>=2:
            value=2
        dict13[key]=value
    return dict13

In [8]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continue
    return continuous_chunk

In [9]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def ispolarityClash(line):
    pos=0
    neg=0
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(line)
    for k in sorted(ss):
        #print('{0}: {1}, '.format(k, ss[k]), end='')
        #print()
        if k=='neg':
            neg=ss[k]
        if k=='pos':
            pos=ss[k]
        if(pos >0 and neg >0):
            return 1
        else:
            return 0

/home/sharath/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [10]:
def istenseclash(sentence):
    print(line)
    text = word_tokenize(sentence)
    tagged = pos_tag(text)
    clash=0
    tense = {}
    tense["future"] = len([word for word in tagged if word[1] == "MD"])
    tense["present"] = len([word for word in tagged if word[1] in ["VBP", "VBZ","VBG"]])
    tense["past"] = len([word for word in tagged if word[1] in ["VBD", "VBN"]]) 
    if tense["past"]>0  and tense["present"]>0 :
        clash=1
    return clash

In [11]:
from nltk.tag import pos_tag
from nltk import ne_chunk
from nltk.tokenize import word_tokenize
from collections import Counter
i=CC_count=0
#count11=[]
syntacticfeatures_count=[]
syntacticfeatures_presence=[]
syntacticfeatures_greater2=[]
syntacticfeatures_ner=[]
for line in input_txt.split('\n'):
    if(len(line)>0):
        count= Counter([j for i,j in pos_tag(word_tokenize(line))])
        #test12=ne_chunk(pos_tag(word_tokenize(line)))
        #print(test12)
        list11=sorted(count.most_common())
        is_ner_present=0
        length_of_ner= len(get_continuous_chunks(line))
        if length_of_ner >0:
            is_ner_present=1
        frequency_ner=length_of_ner/len(line)
        dict11=dict(list11)
        dict13=getCount(dict11)
        dict12=dict.fromkeys(dict11,1)
        tense_clash=istenseclash(line)
        print(tense_clash)
        #count11=getabsoluteCount(dict11,count11)
        syntacticfeatures_count.insert(i,{'CC':dict11.get('CC',0),'JJ':dict11.get('JJ',0),'JJR':dict11.get('JJR',0),'JJS':dict11.get('JJS',0),'MD':dict11.get('MD',0),'NN':dict11.get('NN',0),'NNP':dict11.get('NNP',0),'NNS':dict11.get('NNS',0),'NNPS':dict11.get('NNPS',0),'PRP':dict11.get('PRP',0),'POS':dict11.get('POS',0),'RB':dict11.get('RB',0),'PRP$':dict11.get('PRP$',0),'RBR':dict11.get('RBR',0),'RBS':dict11.get('RBS',0),'UH':dict11.get('UH',0),'VB':dict11.get('VB',0),'VBD':dict11.get('VBD',0),'VBG':dict11.get('VBG',0),'VBP':dict11.get('VBP',0),'VBN':dict11.get('VBN',0),'VBZ':dict11.get('VBZ',0),'WDT':dict11.get('WDT',0),'WP':dict11.get('WP',0),'WP$':dict11.get('WP$',0),'WRB':dict11.get('WRB',0)})
        syntacticfeatures_presence.insert(i,{'CC':dict12.get('CC',0),'JJ':dict12.get('JJ',0),'JJR':dict12.get('JJR',0),'JJS':dict12.get('JJS',0),'MD':dict12.get('MD',0),'NN':dict12.get('NN',0),'NNP':dict12.get('NNP',0),'NNS':dict12.get('NNS',0),'NNPS':dict12.get('NNPS',0),'PRP':dict12.get('PRP',0),'POS':dict12.get('POS',0),'RB':dict12.get('RB',0),'PRP$':dict12.get('PRP$',0),'RBR':dict12.get('RBR',0),'RBS':dict12.get('RBS',0),'UH':dict12.get('UH',0),'VB':dict12.get('VB',0),'VBD':dict12.get('VBD',0),'VBG':dict12.get('VBG',0),'VBP':dict12.get('VBP',0),'VBN':dict12.get('VBN',0),'VBZ':dict12.get('VBZ',0),'WDT':dict12.get('WDT',0),'WP':dict12.get('WP',0),'WP$':dict12.get('WP$',0),'WRB':dict12.get('WRB',0)})
        syntacticfeatures_greater2.insert(i,{'CC':dict13.get('CC',0),'JJ':dict13.get('JJ',0),'JJR':dict13.get('JJR',0),'JJS':dict13.get('JJS',0),'MD':dict13.get('MD',0),'NN':dict13.get('NN',0),'NNP':dict13.get('NNP',0),'NNS':dict13.get('NNS',0),'NNPS':dict13.get('NNPS',0),'PRP':dict13.get('PRP',0),'POS':dict13.get('POS',0),'RB':dict13.get('RB',0),'PRP$':dict13.get('PRP$',0),'RBR':dict13.get('RBR',0),'RBS':dict13.get('RBS',0),'UH':dict13.get('UH',0),'VB':dict13.get('VB',0),'VBD':dict13.get('VBD',0),'VBG':dict13.get('VBG',0),'VBP':dict13.get('VBP',0),'VBN':dict13.get('VBN',0),'VBZ':dict13.get('VBZ',0),'WDT':dict13.get('WDT',0),'WP':dict13.get('WP',0),'WP$':dict13.get('WP$',0),'WRB':dict13.get('WRB',0)})
        syntacticfeatures_ner.insert(i,{'NER_Presence':is_ner_present,'NER_Count':length_of_ner,'Frequency_of_NER':frequency_ner,'TenseClash':tense_clash})
        
        i=i+1
df = pd.DataFrame(syntacticfeatures_count)
df1=pd.DataFrame(syntacticfeatures_presence)
df2=pd.DataFrame(syntacticfeatures_greater2)
df3=pd.DataFrame(syntacticfeatures_ner)
df.to_csv('outfiles/synFeatures_countofTagPerTweet_TestA.csv', sep=',',index=False)
df1.to_csv('outfiles/synFeatures_presenceTaginTweet_TestA.csv', sep=',',index=False)
df2.to_csv('outfiles/synFeatures_greaterthan2inTweet_TestA.csv', sep=',',index=False)
df3.to_csv('outfiles/synFeatures_NamedEntityRecognition_TestA.csv', sep=',',index=False)
#list11=sorted(count.iteritems())
#print(list11)

@someuser Can U Help?More conservatives needed on #TSU + get paid 4 posting stuff like this!YOU $ can go to http://someurl
1
Just walked in to #Starbucks and asked for a "tall blonde" Hahahaha
0
GONNA WIN http://someurl
0
@someuser He is exactly that sort of person. Weirdo!
0
So much at work mate 10/10 #boring 100% #dead mate full on #shit absolutely #sleeping mate can't handle the
0
Corny jokes are my absolute favorite
0
People complain about my backround pic and all I feel is like "hey don't blame me, Albert E might have spoken those words" #life
1
@someuser @someuser Darn, my sock joke needs fixing?
0
if Christian expects Fifa to sleep in my bed with me tonight, he's wrong :angry_face_with_horns:
0
People who tell people with anxiety to "just stop worrying about it" are my favorite kind of people #educateyourself
0
Most important thing I've learned in school http://someurl
1
#OrionLaunch underway - yeah! #Orion http://someurl
0
I love context and large ensemble Fridays!!!!! Der my m

1
The most exciting way to start a Friday: a presentation on structurally deficient bridges. #lobbyinglife
0
New shirt #new #shirt #im #sorry #its #just #i #literally #do #care #at #all http://someurl
0
@someuser @someuser @someuser It wasn't for one person. It was a business expense.
1
HAHAHAHA!! Oops...I'm addicted to @someuser Tweets sooo FUNNY! You Got to follow him for #HILARIOUS #tweets #RT.
0
@someuser @someuser @someuser absolutely loved it! Well done all involved!
1
@someuser That was the case, not the recording.So comparing the two is apple & oranges.
1
Wow...lolNeed cash in your hands today? Go to http://someurl #wun #mca... http://someurl
0
At least I don't have to clean the floor
0
Incidentally the foreword for movie #BhopalAPrayerForRain is by Subroto Roy Sahara - saying - justice delayed is justice denied.
1
@someuser Our 3 losses followed bye weeks & an FCS team. Do we get rid of those next year?
1
#AskSuperwomanLIVE if you could swim in any liquid besides water what wo

@someuser I actually think he needs to use the youngsters in this reincarnation
0
What a golden morning. :unamused_face:
0
Sleep. SLEEP YOU STUPID IDIOT I WANNA LEAVE MY ROOM
0
LIBERAL/#LEFTISTS USE #RACIST/SEXIST #LABEL FOR #DISLIKESMSNBC host Ed Schultz said RACIST to SUPPORT demand for #MINIMUM WAGE#SAD
0
Read at http://someurl the news '#Obama: #The #American #way #of #life #is #a #gift'
1
Really missing my momma today.. More than usual. I don't think today will be a good day
0
@someuser Nevermind that, but do pray tell,would Scherzy still play Grizabella?
0
Everyone wants the truth but nobody wants to be honest...
0
I won't be surprised if shelling across pak border intensifies during Delhi Elections to change issues of election's. http://someurl
0
Every inch of you is perfect from the bottom to the top UNLESS YOU'RE SKINNY #Shade #TheMoreYouKnow
0
Take a look at those calves!!! :smiling_face_with_sunglasses::smiling_face_with_sunglasses::face_with_tears_of_joy::face_with_tears_of

0
So I may have had a "few" empties to recycle... :recycling_symbol::recycling_symbol: I :red_heart: #craftbeer http://someurl
0
@someuser @someuser @someuser @someuser @someuser @someuser yet in Ferguson, MO the mayor and Gov are Dems.
0
Abnormal professor walks in with the finals in a Zoloft bag #psychhumor
0
@someuser in hereford but can't leave bambino alone!
0
@someuser skiing in Regents Park much
0
Rejoice in knowing he is now in his new place... easy
0
When "make dua for me bro" turns into an hour long conversation
0
@_Liam_Walker_ @someuser A thief, a liar, a vote rigger, mocks those who use food banks and an online bully
1
What the hell has happened to Shahid Afridi in last couple of days? Left alone more bouncers than played, got a review right. #pakvnz
1
Sub is making class go by so slow that I feel like I am losing brain cells. #substitute #MoveitAlong
0
http://someurl #Photography #Photoshop #Photos #Art #EyeCandy #Bible on #TSU :person_tipping_hand::Christmas_tree::Santa_

1
Sweet jeebus! MT @someuser CryoSat & Jason-2 confirm very high to phenomenal seas generated by North Atlantic low http://someurl
1
Hyper at 1AM when I really should be getting rest #ohwhale :whale::spouting_whale:
0
We were miles apart,but you found me, now we are close enough, still apart
1
Drinking a Southern Triangle IPA by @someuser at @someuser -- http://someurl
0
American Red Cross van "together we can save lives..." And the driver is smoking.
0
@someuser @someuser It also makes me want to post this... http://someurl
0
#birthcontrol #lol #wtf #whatisbirthcontrol @someuser great commercial
0
Oh how I love working in Baltimore
0
Fans watch #tylerfarr in concert at @someuser #countrymusic #throwdown1035 #tampa http://someurl
0
So I'm the school nurse today. And I have a fever.
0
#BBC #News -- #USA -- #Jeb #Bush to #explore #White #House #bid. http://someurl
0
Thanks for the dick pic, really, thanks.
0
@someuser @someuser spawn of Larry Ellison aka World's Worst Human
1
Really love

0
My entire career has been devoted to being a BOSS in beating corruption, but I get beat in a corrupt election #SoCloseTilden #1Vote
1
Yay I just love this time of the month...!
0
Finished the year strong with a good bike workout! Missed a lot this year resting my shoulder--hoping for better in 2015. Happy new year.
1
@someuser trying hard to be fragile and petite. Hahaha. Happy new year liabeeeeee more clingy moments :face_with_tears_of_joy::face_with_tears_of_joy::face_with_tears_of_joy:
0
@someuser yes, you got the point. :( it's like, as the time passes we're growing into some kind of beasts.
1
Someone took my lucky hat today, yay, isn't that fun... Sarcasm.png #IhopeIGetItBack
1
@someuser grateful for this encouragement :))
0
@someuser @someuser "pit players v. victims?" How dare the NFL put a divide between men and the women and children they beat?
1
@someuser kids don't need schools! Schools just turn them into gay atheists, gaytheists, if you will.
0
@someuser @someuser @someu

Because #LOGICALLY, that does #fully #compute. If they wear #LESS #PROTECTION for a "#DEADLY #VIRUS" & do "#get" #EBOLA as result?
0
Last night watch for 2014!! That deserves :bomb:'r breakfast dram. @someuser 17yr from @someuser #scotch #Islay http://someurl
0
can I just own an isle far away from here where my house is stunningly beautiful with only some closed ppl who appreciates me as a human
1
If only I could be a cooler kid xp lol I'm such a nerd #can #i #know #when #im #gonna #get #rejected
1
@someuser It's so the speaker remembers their talk's over. Otherwise how would they know?
0
Murdered baby, black man in US, kids in Thailand and a woman's body in Avon gorge. It's a jolly day for news today
0
the struggle of writing this article and studying for this journalism final is killer.
0
@someuser @_BARRYISLAND_ Pulled by White Van Man by the looks of it. Yuk .. so naff. That keeps the magic of xmas alive..
0
I'm so sore, work tomorrow is gonna be fun..
1
@someuser just lost a fiver

@someuser that bump off the apron looked like it hurt.. was that padding u landed on?
1
@someuser please make me smile this Xmas.. First Xmas without my mum here. Your my fav everrr. Pleaseeee:Christmas_tree::wrapped_gift::wrapped_gift:
0
@someuser Hope the header didn't mess up that barnet
1
If it's NOT? then it IS! Hand Crafted Artistically Fashionable, Connect @someuser for #fashionblog #thoughts http://someurl
0
It's okay @someuser I didn't really want to see my idol anyway.
1
I'll try to guess the ASCII symbols without my admins help. Always trying to learn.
0
Hearing gun shots when I first wake up makes me feel like I'm in Compton. What a wonderful way to wake up. :red_heart: #imscared
1
Protesters upset over decision in chokehold death take the streets again http://someurl /Mexico's streets aren't the only ones taken
1
@someuser Great,& not sure if i've seen that actual pic! just for girls : D
1
#TGIF oh just kidding I have 30 mins from one job to the next. love 12 hour days. #s